# Phase 1 Notebook: `tools_registry.py`

Target file: `execution/langgraph/tools_registry.py`

Purpose: assemble the tool boundary and wire memoization tools into the same deterministic tool interface.

## 3-Layer Context

- Layer 1 (Directive): strict tool boundaries are part of reliability controls.
- Layer 2 (Orchestration): graph executes tools by name through this registry.
- Layer 3 (Execution): each tool call is deterministic argument->result behavior.

In [ ]:
from pathlib import Path
import sys

def bootstrap_repo_root() -> Path:
    cwd = Path.cwd().resolve()
    candidates = [cwd, *cwd.parents, Path('/home/nir/dev/agent_phase0')]
    for candidate in candidates:
        if (candidate / 'execution' / 'langgraph' / 'tools_registry.py').exists():
            if str(candidate) not in sys.path:
                sys.path.insert(0, str(candidate))
            return candidate
    raise RuntimeError('Could not locate repo root for tools registry notebook.')

repo_root = bootstrap_repo_root()
print('repo_root =', repo_root)
print('kernel_python =', sys.executable)
if '/.venv/' not in sys.executable.replace('\\', '/'):
    print('WARNING: kernel is not the project .venv interpreter.')

## P0 vs P1 (Tool Boundary)

| Concern | Phase 0 (`orchestrator.py` tool dict) | Phase 1 (`tools_registry.py`) | Why this matters |
|---|---|---|---|
| Tool wiring location | inline in orchestrator constructor | dedicated registry module | cleaner separation of concerns |
| Memoization integration | simple file memoize tool | store-backed memoize + retrieval tools | reusable memory interface and testing |
| Extensibility | edits in orchestrator | add/swap tools in one place | safer iteration |

## Why this block exists: inspect registry and memo tools

We inspect local tool wrappers to confirm run-scoped memo args are enforced and registry shape is stable.

In [ ]:
import inspect
import execution.langgraph.tools_registry as registry_mod

print(inspect.getsource(registry_mod.MemoizeStoreTool))
print(inspect.getsource(registry_mod.RetrieveMemoTool))
print(inspect.getsource(registry_mod.build_tool_registry))

## Why this block exists: registry demo

This ensures all expected tool names are present and memo put/get flow works end-to-end through tool interfaces.

In [ ]:
from execution.langgraph.memo_store import SQLiteMemoStore
from execution.langgraph.tools_registry import build_tool_registry

store = SQLiteMemoStore('.tmp/notebook_demo_registry.db')
tools = build_tool_registry(store)
sorted(tools.keys())

In [ ]:
memo_result = tools['memoize'].execute({
    'run_id': 'run-registry-demo',
    'key': 'write_file:fib.txt',
    'value': {'ok': True},
    'source_tool': 'write_file',
    'step': 3
})
lookup_result = tools['retrieve_memo'].execute({
    'run_id': 'run-registry-demo',
    'key': 'write_file:fib.txt'
})
memo_result, lookup_result

In [ ]:
expected = {'repeat_message', 'sort_array', 'string_ops', 'math_stats', 'write_file', 'memoize', 'retrieve_memo'}
assert expected.issubset(set(tools.keys()))
assert lookup_result['found'] is True
print('tools registry assertions passed')

## Takeaways

- Registry centralizes tool boundary concerns for P1 orchestration.
- Memoization tools are now first-class peers in the toolset.
- P0 inline mapping is preserved conceptually but modularized for scale.
- Next notebook: `execution/notebooks/p1_graph_orchestrator.ipynb`.